In [41]:
# Initial imports
import requests
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [42]:
# Create a Dataframe with crypto Data
file_path = Path("Clustering_Crypto_Data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [43]:
#Data PreProcessing

In [44]:
# Keep only necessary columns: 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [45]:
# Keep only cryptocurrencies that are on trading
crypto_df = crypto_df[crypto_df.IsTrading == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [46]:
# Keep only cryptocurrencies with a working algorithm
crypto_df['Algorithm'].dropna()
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [47]:
# Remove the "IsTrading" column
crypto_df.drop(columns = ["IsTrading"],inplace = True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [48]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace = True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [49]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace = True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [50]:
# Remove rows with cryptocurrencies with no coins mined
crypto_df = crypto_df[crypto_df.TotalCoinsMined != 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [51]:
# Fetch the cryptocurrencies names prior to drop them from crypto_df
coin_name = crypto_df['CoinName']

In [52]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df.loc[crypto_df.CoinName != "N/A"]
crypto_df = crypto_df.loc[crypto_df.ProofType != "N/A"]
crypto_df = crypto_df.loc[crypto_df.TotalCoinsMined != "N/A"]
crypto_df = crypto_df.loc[crypto_df.TotalCoinSupply != "N/A"]
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [53]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinName_df = pd.DataFrame({'CoinName': crypto_df['CoinName']}) 
coinName_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [54]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coins_name = crypto_df.CoinName
coins_name.head()

42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
Name: CoinName, dtype: object

In [55]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
df = crypto_df.drop(columns = "CoinName")
df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [56]:
# Create dummy variables for text features
df_dummy = pd.get_dummies(df["Algorithm"])
df_dummy = pd.get_dummies(df["ProofType"])
df_dummy.head()

,DPOS,DPoC,DPoS,HPoW,LPoS,POBh,PoA,PoC,PoS,PoS/LPoS,...,PoW/PoS,PoW/PoS,PoW/PoW,PoW/nPoS,Pos,Proof of Authority,Proof of Trust,TPoS,Zero-Knowledge Proof,dPoW/PoW
42,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
# Standardize data
df_scaled = StandardScaler().fit_transform(df_dummy)
df_scaled[0:3]

array([[-0.0433555 , -0.0433555 , -0.13105561, -0.0433555 , -0.0433555 ,
        -0.0433555 , -0.0433555 , -0.07523548, -0.4386271 , -0.0433555 ,
        -0.06137164, -0.0433555 , -0.0433555 , -0.89480483, -0.0433555 ,
        -0.0433555 ,  1.42422228, -0.0433555 , -0.0433555 , -0.0433555 ,
        -0.0433555 , -0.0433555 , -0.0433555 , -0.0433555 , -0.0433555 ,
        -0.0433555 ],
       [-0.0433555 , -0.0433555 , -0.13105561, -0.0433555 , -0.0433555 ,
        -0.0433555 , -0.0433555 , -0.07523548, -0.4386271 , -0.0433555 ,
        -0.06137164, -0.0433555 , -0.0433555 , -0.89480483, -0.0433555 ,
        -0.0433555 ,  1.42422228, -0.0433555 , -0.0433555 , -0.0433555 ,
        -0.0433555 , -0.0433555 , -0.0433555 , -0.0433555 , -0.0433555 ,
        -0.0433555 ],
       [-0.0433555 , -0.0433555 , -0.13105561, -0.0433555 , -0.0433555 ,
        -0.0433555 , -0.0433555 , -0.07523548, -0.4386271 , -0.0433555 ,
        -0.06137164, -0.0433555 , -0.0433555 , -0.89480483, -0.0433555 ,
       

In [58]:
#Reducing Dimensions Using PCA

In [59]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
pca = pca.fit_transform(df_scaled)

In [60]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(data = pca, 
                      columns=["PC 1", "PC 2", "PC 3"], index = crypto_df.index)
pcs_df.head()

,PC 1,PC 2,PC 3
42,1.518440,-0.828022,0.093803
404,1.518440,-0.828022,0.093803
1337,1.518440,-0.828022,0.093803
BTC,-1.352768,-0.344062,0.057973
ETH,-1.352768,-0.344062,0.057973


In [61]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [62]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["class"] = model.labels_
pcs_df.head()

,PC 1,PC 2,PC 3,class
42,1.518440,-0.828022,0.093803,0
404,1.518440,-0.828022,0.093803,0
1337,1.518440,-0.828022,0.093803,0
BTC,-1.352768,-0.344062,0.057973,1
ETH,-1.352768,-0.344062,0.057973,1


In [63]:
#Visualizing the results

In [64]:
# Creating a new Dataframe
clustered_df = pd.DataFrame({
    "Algorithm" : crypto_df.Algorithm,
    "ProofType": crypto_df.ProofType,
    "TotalCoinsMined": crypto_df.TotalCoinsMined,
    "TotalCoinSupply": crypto_df.TotalCoinSupply,
    "PC 1": pcs_df["PC 1"],
    "PC 2": pcs_df["PC 2"], 
    "PC 3": pcs_df["PC 3"],
    "CoinName": coins_name,
    "Class": pcs_df["class"]
    }, index = crypto_df.index)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,1.518440,-0.828022,0.093803,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,1.518440,-0.828022,0.093803,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,1.518440,-0.828022,0.093803,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-1.352768,-0.344062,0.057973,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-1.352768,-0.344062,0.057973,Ethereum,1


In [65]:
# Create a 3D-Scatter with the PCA data and the clusters
#It showed the first time I ran it on my notebook, but after rerunning the notebook a few times it doesnt come up anymore.
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [66]:
# Table with tradable cryptos
display(crypto_df)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [67]:
# Print the total number of tradable cryptocurrencies
num_of_tradable_cryptocurrencies = len(crypto_df)

print('The total number of tradable cryptocurrencies is:')
print(num_of_tradable_cryptocurrencies)

The total number of tradable cryptocurrencies is:
533


In [68]:
# Scale data to create the scatter plot
clustered_df['TotalCoinsMined'] = clustered_df['TotalCoinsMined'].astype(float) / 100000000

clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype(float) / 100000000

In [70]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot(
    kind="scatter", 
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    c='Class',
    rot= 90, 
    colormap="inferno", 
    hover_cols=['CoinName'])

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,Class,CoinName)